In [1]:
from dask_jobqueue import SLURMCluster

In [52]:
cluster = SLURMCluster(cores=4, memory='32GB', queue = 'short',walltime='10:00:00')

Task exception was never retrieved
future: <Task finished name='Task-175500' coro=<Client._gather.<locals>.wait() done, defined at /home/igarzonalva/.conda/envs/arboreto-env/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/home/igarzonalva/.conda/envs/arboreto-env/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit


In [58]:
cluster

SLURMCluster(65fd7238, 'tcp://159.237.145.53:44251', workers=4, threads=4, memory=29.80 GiB)

In [ ]:
cluster.scale(10)

In [57]:
from dask.distributed import Client

In [59]:
client = Client('tcp://159.237.145.53:44251')

In [60]:
import scanpy as sc

In [61]:
adata = sc.read_h5ad('/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/02_Integration/adata/adata_scanvi_cuda_refinement.h5ad')

In [62]:
adata = adata[adata.obs.subtype == 'TNBC',:].copy()

In [63]:
import pandas as pd
from arboreto.algo import grnboost2
from arboreto.utils import load_tf_names

In [64]:
expression_mat = pd.DataFrame(adata.X.toarray(), index=adata.obs_names, columns=adata.var_names)
TF_FNAME = "/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/04_grn/Common_files/TF_names_v_1.01.txt"
tf_names = load_tf_names(TF_FNAME)

In [65]:
network = grnboost2(
                expression_data=expression_mat,
                tf_names=tf_names,
                client_or_address=client,
                verbose = True,)

preparing dask client
parsing input
creating dask graph
8 partitions
computing dask graph


/home/igarzonalva/.conda/envs/arboreto-env/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 1.45 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


not shutting down client, client was created externally
finished


KeyboardInterrupt: 

In [ ]:
cluster.close()